# 2.3 - Comprobación localización

Mi manera de comprabar esto es dejar que sea el modelo el me diga, con las métricas de error que forma de transformar el dato es mejor, si tener la tupla latitud-longitud, la transformación esférica que realizé anteriormente con las variables x-y-z o realizar un one-hot encoding directamente con los barrios o distritos.  

In [1]:
# librerias

import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
cols=['neighbourhood_cleansed','neighbourhood_group_cleansed', 'accommodates', 
      'air_conditioning', 'availability_30', 'availability_365', 'availability_60', 'availability_90',
      'bathrooms', 'bedrooms', 'beds', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
      'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 'cleaning_fee',
      'dishwasher', 'dryer', 'elevator', 'extra_people', 'guests_included', 'latitude', 'longitude', 'maximum_nights',
      'minimum_nights', 'number_of_reviews', 'number_of_reviews_ltm', 'room_type_private_room', 'room_type_shared_room',
      'security_deposit', 'x', 'y', 'z', 'price']

In [3]:
# carga de datos
listings=pd.read_csv('../data/transform_data/listings_normal.csv', usecols=cols)

listings=listings[(listings.price>=10) & (listings.price<=196)]  # eliminacion de outliers

# cambio en el tamaño del tipo de dato
for c in listings.select_dtypes(include='int'):
    listings[c]=pd.to_numeric(listings[c], downcast='integer')

for c in listings.select_dtypes(include='float'):
    listings[c]=pd.to_numeric(listings[c], downcast='float')
    
listings.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18936 entries, 0 to 21311
Data columns (total 34 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   neighbourhood_cleansed                        18936 non-null  object 
 1   neighbourhood_group_cleansed                  18936 non-null  object 
 2   latitude                                      18936 non-null  float32
 3   longitude                                     18936 non-null  float32
 4   accommodates                                  18936 non-null  int8   
 5   bathrooms                                     18936 non-null  int8   
 6   bedrooms                                      18936 non-null  int8   
 7   beds                                          18936 non-null  int8   
 8   price                                         18936 non-null  int16  
 9   security_deposit                              18936 non-null 

In [4]:
listings.head()

,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,bedrooms,beds,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,availability_30,availability_60,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,x,y,z,room_type_private_room,room_type_shared_room,dryer,air_conditioning,elevator,dishwasher
0,Hispanoamérica,Chamartín,40.456280,-3.67763,2,1,1,0,70,0,5,2,15,1,365,22,52,82,82,73,14,1,0,1,0,0.796991,-0.473457,0.375026,1,0,0,1,1,0
1,Cármenes,Latina,40.403412,-3.74084,1,1,1,1,17,0,0,1,8,4,40,0,0,0,162,33,0,2,0,2,0,0.748058,-0.510948,0.423488,1,0,0,1,1,0
2,Legazpi,Arganzuela,40.386951,-3.69304,6,2,3,5,50,300,30,1,10,15,730,2,2,2,213,0,0,6,6,0,0,0.765575,-0.470904,0.438342,0,0,1,1,1,1
3,Universidad,Centro,40.422020,-3.70395,3,1,0,1,80,200,30,2,0,5,730,27,57,87,362,9,7,1,1,0,0,0.772927,-0.487128,0.406559,0,0,0,1,1,0
4,Justicia,Centro,40.419949,-3.69764,4,1,2,3,115,200,0,4,0,3,10,24,54,77,342,67,15,1,1,0,0,0.775268,-0.481796,0.408451,0,0,0,1,1,0


**Preparación de los 4 paquete de datos**

In [5]:
df_xyz=listings.drop(columns=['latitude', 'longitude', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'])

df_lat_lon=listings.drop(columns=['x', 'y', 'z', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'])

df_barrio=listings.drop(columns=['x', 'y', 'z', 'latitude', 'longitude', 'neighbourhood_group_cleansed'])
df_barrio=pd.get_dummies(df_barrio, prefix='', prefix_sep='')

df_distrito=listings.drop(columns=['x', 'y', 'z', 'latitude', 'longitude', 'neighbourhood_cleansed'])
df_distrito=pd.get_dummies(df_distrito, prefix='', prefix_sep='')

In [6]:
df_xyz.shape

(18936, 30)

In [7]:
df_lat_lon.shape

(18936, 29)

In [8]:
df_barrio.shape

(18936, 154)

In [9]:
df_distrito.shape

(18936, 48)

In [10]:
data={'esféricas': df_xyz, 'latitud-longitud': df_lat_lon, 
      'barrio': df_barrio, 'distrito': df_distrito}

### Testeo de varios modelos en varios paquetes de datos

In [11]:
from sklearn.ensemble import RandomForestRegressor as RFR

from xgboost import XGBRegressor as XGBR

from lightgbm import LGBMRegressor as LGBMR

from catboost import CatBoostRegressor as CTR

from sklearn.model_selection import train_test_split as tts 

from sklearn.metrics import mean_squared_error as mse 
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score as r2

from tqdm.notebook import tqdm

In [12]:
def test_model_data(modelo: object, data: pd.DataFrame, dataname: str) -> str:
    
    X=data.drop('price', axis=1)
    y=data.price

    X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, test_size=0.2, random_state=42)
    
    modelo.fit(X_train, y_train)
    
    y_pred=modelo.predict(X_test)
    
    print(dataname)
    print(f'RMSE: {mse(y_test, y_pred, squared=False)}')
    print(f'MAE: {mae(y_test, y_pred)}')
    print(f'R2: {r2(y_test, y_pred)}')
    print()
    return 'Done!'

In [13]:
%%time

print('\n\033[1m' + 'Prueba Random Forest Regressor' + '\033[0m')

[test_model_data(RFR(), d, n) for n,d in data.items()]


Prueba Random Forest Regressor
esféricas
RMSE: 23.24767402780918
MAE: 15.735667898627243
R2: 0.6691465614678902

latitud-longitud
RMSE: 23.076845915268475
MAE: 15.695472544878566
R2: 0.6739910385710364

barrio
RMSE: 23.53675210434964
MAE: 15.695214894361476
R2: 0.6608672713574222

distrito
RMSE: 23.313629987215016
MAE: 15.73503815935033
R2: 0.6672665702757302

CPU times: user 35.2 s, sys: 135 ms, total: 35.3 s
Wall time: 35.3 s


['Done!', 'Done!', 'Done!', 'Done!']

In [14]:
%%time

print('\n\033[1m' + 'Prueba XG Boosting Regressor' + '\033[0m')

[test_model_data(XGBR(), d, n) for n,d in data.items()]


Prueba XG Boosting Regressor
esféricas
RMSE: 22.45243263244629
MAE: 15.239723205566406
R2: 0.6913946700872258

latitud-longitud
RMSE: 22.41477394104004
MAE: 15.222554206848145
R2: 0.6924289829426343

barrio
RMSE: 22.867399215698242
MAE: 15.5220308303833
R2: 0.6798819349636882

distrito
RMSE: 22.592540740966797
MAE: 15.457427978515625
R2: 0.6875310910456449

CPU times: user 25.7 s, sys: 4.53 s, total: 30.2 s
Wall time: 4.05 s


['Done!', 'Done!', 'Done!', 'Done!']

In [15]:
%%time

print('\n\033[1m' + 'Prueba LightGBM Regressor' + '\033[0m')

[test_model_data(LGBMR(), d, n) for n,d in data.items()]


Prueba LightGBM Regressor
esféricas
RMSE: 22.27956413201295
MAE: 15.242575849575124
R2: 0.6961284660669169

latitud-longitud
RMSE: 22.395638077371125
MAE: 15.34068523213501
R2: 0.6929539472114248

barrio
RMSE: 22.669572259463457
MAE: 15.561534089252865
R2: 0.6853966888817127

distrito
RMSE: 22.511754378759456
MAE: 15.395866128479799
R2: 0.6897617650218114

CPU times: user 1.45 s, sys: 1.5 s, total: 2.95 s
Wall time: 1.44 s


['Done!', 'Done!', 'Done!', 'Done!']

In [16]:
%%time

print('\n\033[1m' + 'Prueba Catboost Regressor' + '\033[0m')

[test_model_data(CTR(verbose=0), d, n) for n,d in data.items()]


Prueba Catboost Regressor
esféricas
RMSE: 21.620325517639113
MAE: 14.708807872358754
R2: 0.7138451608820328

latitud-longitud
RMSE: 21.596826764285495
MAE: 14.774018559744613
R2: 0.7144668562147605

barrio
RMSE: 22.230216236277382
MAE: 15.13325300155063
R2: 0.6974730896607023

distrito
RMSE: 21.944160251765993
MAE: 14.9230074810349
R2: 0.7052087626753057

CPU times: user 19.2 s, sys: 9.62 s, total: 28.8 s
Wall time: 8.64 s


['Done!', 'Done!', 'Done!', 'Done!']

In [21]:
print('\n\033[1m' + 'Catboost Regressor' + '\033[0m')

test_model_data(CTR(verbose=0), df_xyz, 'esféricas')


Catboost Regressor
esféricas
RMSE: 21.620325517639113
MAE: 14.708807872358754
R2: 0.7138451608820328



'Done!'

Este último modelo es funciona mejor en términos de RMSE y MAE que usando la latitud-longitud, y mejor que usando las variables dummies. Esta será la selección final de variables y también el modelo que vamos a usar. En el próximo notebook entrenaré el modelo, ajustaré hiperparámetros y evaluaré.